## Image classification using ResNet50

### Prepare model and test data

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from PIL import Image as PilImage

2024-06-17 11:13:11.002772: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-17 11:13:11.013831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:/usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:
2024-06-17 11:13:11.013895: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


We recommend using `Image` to represent a batch of images. `Image` can be constructed from a numpy array or a Pillow image. The following code loads one test image and resizes them to (224, 224).

In [2]:
# Load the test image
img_raw = PilImage.open('../data/image/jellyfish.png').convert('RGB').resize((224,224))
img = tf.keras.preprocessing.image.img_to_array(img_raw)
tf.keras.applications.resnet.preprocess_input(img)
# img = img[np.newaxis,:,:,:]
# Load the class names
with open('../data/image/imagenet_class_index.json', 'r') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]

In [3]:
model = ResNet50(include_top=True, weights='imagenet', classes=1000)

2024-06-17 11:13:19.015453: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:/usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:
2024-06-17 11:13:19.015701: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:/usr/local/cuda/cudnnv895/lib:/usr/local/cuda/cudnnv895/include:/usr/local/cuda/lib64:
2024-06-17 11:13:19.015835: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublas

The preprocessing function takes an `Image` instance as its input and outputs the processed features that the ML model consumes. In this example, each image in the `Image` object is transformed via `tf.keras.preprocessing.image.img_to_array` followed by `resnet.preprocess_input`.

In [4]:
# The preprocessing function
def resnet_fn(array):
    model = ResNet50(include_top=True, weights='imagenet', classes=1000)
    return model.predict(array)

Let's check the predicted labels of this test image.

In [5]:
predictions = model.predict(img[np.newaxis,:,:,:])
top_indices = np.argsort(predictions)[0, ::-1][:5]
r = tuple((i, idx2label[i]) for i in top_indices)
print(r)

1/1 [==============================] - 2s 2s/step
((107, 'jellyfish'), (108, 'sea_anemone'), (393, 'anemone_fish'), (117, 'chambered_nautilus'), (1, 'goldfish'))


Load background images for algorithms like SHAP, LIME. Randomly sampled from imagenet training set.

In [6]:
import glob
import os
# Load the test image
png_files = glob.glob(os.path.join('/home/kxlu/Documents/xai_compare/data/image/imagenet_1000','*.png'))
bg_img_list = []
for png_file in png_files:
    bg_img_list.append(PilImage.open(png_file).convert('RGB').resize((224,224)))

### AIX360: Protodash

In [7]:
from aix360.algorithms.protodash import ProtodashExplainer


explainer = ProtodashExplainer()
# Explain the top label
explanations = explainer.explain_instance(img)
explanations.ipython_plot(index=0, class_names=idx2label)

ModuleNotFoundError: No module named 'tensorflow.contrib'